In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


# Start to test some model

## import of differrent sample of data set

In [2]:
active_users_df_10PlusRatings_partial = pd.read_csv('../raw_data/active_users_df_10PlusRatings_partial.csv')

In [3]:
active_users_df_100PlusRatings_partial = pd.read_csv('../raw_data/active_users_df_100PlusRatings_partial.csv')

In [4]:
rating_complete_df_10PlusRatings_partial = pd.read_csv('../raw_data/rating_complete_df_10PlusRatings_partial.csv')

In [5]:
rating_complete_df_100PlusRatings_partial = pd.read_csv('../raw_data/rating_complete_df_100PlusRatings_partial.csv')

In [6]:
anime_df = pd.read_csv('../raw_data/anime.csv')

## data set normalisation

In [7]:
active_users_df_10PlusRatings_partial['rating'] = active_users_df_10PlusRatings_partial['rating']/10

In [8]:
active_users_df_100PlusRatings_partial['rating'] = active_users_df_100PlusRatings_partial['rating']/10

In [9]:
rating_complete_df_10PlusRatings_partial['rating'] = rating_complete_df_10PlusRatings_partial['rating']/10

In [10]:
rating_complete_df_100PlusRatings_partial['rating'] = rating_complete_df_100PlusRatings_partial['rating']/10

## data set merge

In [11]:
anime_name_df = anime_df[['MAL_ID','Name']].rename(columns={'MAL_ID' : 'anime_id'})

In [12]:
active_users_df_10PlusRatings_partial_merge = active_users_df_10PlusRatings_partial.merge(anime_name_df, on = 'anime_id', how='inner')

In [13]:
active_users_df_100PlusRatings_partial_merge = active_users_df_100PlusRatings_partial.merge(anime_name_df, on = 'anime_id', how='inner')

In [14]:
rating_complete_df_10PlusRatings_partial_merge = rating_complete_df_10PlusRatings_partial.merge(anime_name_df, on = 'anime_id', how='inner')

In [15]:
rating_complete_df_100PlusRatings_partial_merge = rating_complete_df_100PlusRatings_partial.merge(anime_name_df, on = 'anime_id', how='inner')

## data set vectorization

In [21]:
active_users_df_10PlusRatings_partial_pivot = active_users_df_10PlusRatings_partial_merge.pivot_table(index='Name',columns='user_id',values='rating').fillna(0)

In [22]:
active_users_df_100PlusRatings_partial_pivot = active_users_df_100PlusRatings_partial_merge.pivot_table(index='Name',columns='user_id',values='rating').fillna(0)

In [23]:
active_users_df_10PlusRatings_partial_pivot

user_id,0,1,2,3,4,5,6,7,8,11,...,4987,4988,4989,4990,4991,4992,4993,4996,4997,4998
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xxxHOLiC Rou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0
xxxHOLiC Shunmuki,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
active_users_df_100PlusRatings_partial_pivot

user_id,1,3,4,6,11,14,16,17,19,21,...,4983,4986,4987,4989,4990,4991,4992,4993,4996,4998
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xxxHOLiC Rou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0
xxxHOLiC Shunmuki,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#pca can do at this step but on a sample data set

## KNN

### for active_users_df_10PlusRatings_partial_pivot

In [26]:
#test model knn for users
active_users_df_10PlusRatings_partial_pivot_matrix = csr_matrix(active_users_df_10PlusRatings_partial_pivot.values)

model_knn_10PlusRatings = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn_10PlusRatings.fit(active_users_df_10PlusRatings_partial_pivot_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [28]:
query_index = np.random.choice(active_users_df_10PlusRatings_partial_pivot.shape[0])
print(query_index)
distances, indices = model_knn_10PlusRatings.kneighbors(active_users_df_10PlusRatings_partial_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

11874


### active_users_df_100PlusRatings_partial_pivot

In [38]:
#test model knn for users
active_users_df_100PlusRatings_partial_pivot_matrix = csr_matrix(active_users_df_100PlusRatings_partial_pivot.values)

model_knn_100PlusRatings = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn_100PlusRatings.fit(active_users_df_100PlusRatings_partial_pivot_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [39]:
query_index = np.random.choice(active_users_df_100PlusRatings_partial_pivot.shape[0])
print(query_index)
distances, indices = model_knn_100PlusRatings.kneighbors(active_users_df_100PlusRatings_partial_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

6898


### Predictions and comparaisons of model

In [80]:
# don't forget to when the distance for knn prediction is 0.0 is probably due lack of data and 
# not an extreme proximity of the anime

In [44]:
#chose a random anime
query_index = np.random.choice(active_users_df_10PlusRatings_partial_pivot.shape[0])
print(query_index)
distances, indices = model_knn_10PlusRatings.kneighbors(active_users_df_10PlusRatings_partial_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

9521


In [77]:
#Chose the anime 5 who is the same on the two vectorized data
query_index = 25
n_neighbors = 15
print(query_index)
distances, indices = model_knn_10PlusRatings.kneighbors(active_users_df_10PlusRatings_partial_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = n_neighbors)

25


In [78]:
#10PlusRatings
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(active_users_df_10PlusRatings_partial_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, active_users_df_10PlusRatings_partial_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for .hack//The Movie: Sekai no Mukou ni:

1: .hack//Versus: The Thanatos Report, with distance of 0.48661959563165813:
2: .hack//Quantum, with distance of 0.5784280285798832:
3: .hack//G.U. Trilogy, with distance of 0.5939294169716975:
4: .hack//G.U. Returner, with distance of 0.6755530612638028:
5: Zenonzard The Animation, with distance of 0.6769918613135156:
6: .hack//Roots, with distance of 0.6939417625413009:
7: Genei wo Kakeru Taiyou: Fumikomenai Kokoro, with distance of 0.695664278478302:
8: .hack//G.U. Trilogy: Parody Mode, with distance of 0.7001814758247107:
9: Kiddy Grade: Ignition, with distance of 0.7088983815444148:
10: Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou, with distance of 0.7098601342525508:
11: Zenonzard The Animation Episode 0, with distance of 0.7138356018686633:
12: Dae Jang Geum: Jang Geum's Dream 2, with distance of 0.7217682522013691:
13: .hack//Unison, with distance of 0.7236125889483194:
14: Circuit Angel: Ketsui no Starting Grid, wi

In [74]:
print(query_index)
distances, indices = model_knn_10PlusRatings.kneighbors(active_users_df_10PlusRatings_partial_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = n_neighbors)

25


In [79]:
#100PlusRatings
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(active_users_df_100PlusRatings_partial_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, active_users_df_100PlusRatings_partial_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for .hack//The Movie: Sekai no Mukou ni:

1: .hack//Versus: The Thanatos Report, with distance of 0.48661959563165813:
2: .hack//Quantum, with distance of 0.5784280285798832:
3: .hack//G.U. Trilogy, with distance of 0.5939294169716975:
4: .hack//G.U. Returner, with distance of 0.6755530612638028:
5: Zettai Shougeki: Platonic Heart, with distance of 0.6769918613135156:
6: .hack//Roots, with distance of 0.6939417625413009:
7: Generator Gawl, with distance of 0.695664278478302:
8: .hack//G.U. Trilogy: Parody Mode, with distance of 0.7001814758247107:
9: Kidou Keisatsu Patlabor: On Television, with distance of 0.7088983815444148:
10: Yuu☆Yuu☆Hakusho Picture Drama, with distance of 0.7098601342525508:
11: Zettai Shounen, with distance of 0.7138356018686633:
12: Dagram vs. Round-Facer, with distance of 0.7217682522013691:
13: .hack//Unison, with distance of 0.7236125889483194:
14: Citrus, with distance of 0.7273831737235268:
